# spaCy NLP

Notebook for evaluating the functionality of the NLP framework spaCy

Comparison with further frameworks:
[Comparison of Top 6 Python NLP Libraries](https://medium.com/activewizards-machine-learning-company/comparison-of-top-6-python-nlp-libraries-c4ce160237eb)

- Tokenization
- Named Entity Recognition
- Word Vectors and Similarity
- Integration with sklearn

Author: Enrico Lauckner ([github.com/elauckne](github.com/elauckne))

In [11]:
import spacy
from spacy import displacy

### Configure spaCy

Load Language Model for German

In [2]:
# Run as administrator in terminal
! python -m spacy download de_core_news_sm


    Linking successful
    C:\Users\elauckner\Anaconda3\lib\site-packages\de_core_news_sm -->
    C:\Users\elauckner\Anaconda3\lib\site-packages\spacy\data\de_core_news_sm

    You can now load the model via spacy.load('de_core_news_sm')



Ihre Berechtigungen reichen nicht aus, um diesen Vorgang auszuführen.


In [3]:
nlp = spacy.load('de_core_news_sm')
nlp 

### Tokenization

Part-of-speech tags and dependencies

* Text: The original word text.
* Lemma: The base form of the word.
* POS: The simple part-of-speech tag.
* Tag: The detailed part-of-speech tag.
* Dep: Syntactic dependency, i.e. the relation between tokens.
* Shape: The word shape – capitalisation, punctuation, digits.
* is alpha: Is the token an alpha character?
* is stop: Is the token part of a stop list, i.e. the most common words of the language?

In [4]:
text = 'Eine große Welle an Berichten schwappte von Freitag an durch alle Medien.'
doc = nlp(text)

In [5]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

Eine Eine DET ART nk Xxxx True False
große groß ADJ ADJA nk xxxx True True
Welle Welle NOUN NN sb Xxxxx True False
an an ADP APPR mnr xx True True
Berichten Bericht NOUN NN nk Xxxxx True False
schwappte schwappen VERB VVFIN ROOT xxxx True False
von von ADP APPR mo xxx True True
Freitag Freitag NOUN NN nk Xxxxx True False
an an ADP APZR ac xx True True
durch durch ADP APPR mo xxxx True True
alle all DET PIAT nk xxxx True True
Medien Medium NOUN NN nk Xxxxx True False
. . PUNCT $. punct . False False


In [6]:
spacy.explain("DET")

'determiner'

In [7]:
spacy.explain("nk")

'noun kernel element'

### Named Entity Recognition

In [8]:
text_ent = 'Emanuel Macron (Präsident von Frankreich) trifft den Geschäftsführer von Microsoft.'
doc = nlp(text_ent)

In [9]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Emanuel Macron 0 14 PER
Frankreich 30 40 LOC
Microsoft 73 82 ORG


Visualize entities, result is displayed on http://localhost:5000/

In [12]:
displacy.serve(doc, style='ent')


    Serving on port 5000...
    Using the 'ent' visualizer


    Shutting down server on port 5000.



### Word Vectors and Similarity

In [13]:
tokens = nlp('Geldwäsche Euro Bareinzahlung Betrug')

for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

Geldwäsche Geldwäsche 1.0
Geldwäsche Euro 0.5938709
Geldwäsche Bareinzahlung 0.49834657
Geldwäsche Betrug 0.49147055
Euro Geldwäsche 0.5938709
Euro Euro 1.0
Euro Bareinzahlung 0.7818639
Euro Betrug 0.6534954
Bareinzahlung Geldwäsche 0.49834657
Bareinzahlung Euro 0.7818639
Bareinzahlung Bareinzahlung 1.0
Bareinzahlung Betrug 0.6406553
Betrug Geldwäsche 0.49147055
Betrug Euro 0.6534954
Betrug Bareinzahlung 0.6406553
Betrug Betrug 1.0


### Integration with sklearn

[Tutorial](https://www.analyticsvidhya.com/blog/2017/04/natural-language-processing-made-easy-using-spacy-%E2%80%8Bin-python/)

Sentiment Classification Sample Data:

In [14]:
train = [('Ich liebe dieses Sandwich.', 'pos'),          
         ('das war ein toller ort!', 'pos'),
         ('ich mochte die Biere.', 'pos'),
         ('das ist meine beste Arbeit.', 'pos'),
         ("was für eine großartige aussicht", 'pos'),
         ('Ich mag dieses Restaurant nicht', 'neg'),
         ('Ich hab genug von dem Zeug.', 'neg'),
         ("Ich komm damit nicht klar", 'neg'),
         ('Er ist mein Erzfeind!', 'neg'),          
         ('mein chef ist schrecklich.', 'neg')] 
test =   [('das bier war gut.', 'pos'),     
         ('meine arbeit macht mir keinen spaß', 'neg'),
         ("ich fühle mich nicht dandy heute.", 'neg'),
         ("ich fühle micht nicht gut!", 'neg'),
         ('Gary ist mein bester freund.', 'pos'),
         ("Ich kann nciht glauben, dass ich das tue.", 'neg')]

In [15]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score 

from spacy.lang.de import German
from spacy.lang.de.stop_words import STOP_WORDS as stopwords_ger

import string

In [16]:
parser = German()
punctuations = string.punctuation

Create spacy tokenizer that parses a sentence and generates tokens

In [17]:
def spacy_tokenizer(sentence):
    tokens = parser(sentence)
    tokens = [tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_ for tok in tokens]
    tokens = [tok for tok in tokens if (tok not in stopwords_ger and tok not in punctuations)]     
    return tokens

Set up pipeline

In [18]:
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,2))
classifier = LinearSVC()

pipe = Pipeline([('vectorizer', vectorizer),
                 ('classifier', classifier)])

In [19]:
pipe.fit([x[0] for x in train], [x[1] for x in train]) 
pred_data = pipe.predict([x[0] for x in test]) 
for (sample, pred) in zip(test, pred_data):
    print(sample, pred )

('das bier war gut.', 'pos') neg
('meine arbeit macht mir keinen spaß', 'neg') pos
('ich fühle mich nicht dandy heute.', 'neg') neg
('ich fühle micht nicht gut!', 'pos') neg
('Gary ist mein bester freund.', 'pos') neg
('Ich kann nciht glauben, dass ich das tue.', 'neg') neg


In [20]:
print("Accuracy:", accuracy_score([x[1] for x in test], pred_data))

Accuracy: 0.3333333333333333
